# Mapping and SNP Calling Tutorial

High-throughput sequencing technologies have in the past few years been producing millions of reads of human genome and other species. To be useful, this genetic information has to be 'put together' in a smart way, in the same way as the pieces of a puzzle (reads) need to be mounted according to a picture (reference genome). 

The present tutorial, like the rest of the course material, is available at our [open-source github repository](https://github.com/hds-sandbox/Popgen_course_aarhus), and is based on [Kasper Munk's course repository](https://github.com/kaspermunch/PopulationGenomicsCourse).

## Content

In this exercise section you will be exposed to different softwares used for mapping reads to a reference sequence and calling variants from the produced alignments. We will use a dataset composed of 28 individuals from 3 different regions: Africa, EastAsia and WestEurasia. You can find the metadata file, containing sample IDs and some extra information for each sample here: `../../Data/metadata/Sample_meta_subset.tsv`

![](img/unnamed-chunk-1-1.png)

This dataset is a subset of the [Simons Diversity Project](https://www.nature.com/articles/nature18964).
The following tutorial is based on the individual **S_Ju_hoan_North-3**, which is an african sample from Namibia.

## How to make this notebook work

* In this notebook we will use primarily `R` and `command line bash` commands.
* This notebook integrates multiple coding languages. This means you need to choose a kernel every time we shift from one language to another. A kernel contains a programming language and the necessary packages to run the course material. To choose a kernel, go on the menu on the top of the page and select `Kernel --> Change Kernel`, and then select the preferred one. We will shift between two kernels, and along the code in this notebook you will see a picture telling you to change kernel. The two pictures are below:

<img src="img/bash.png" alt="Bash" width="80"> Shift to the `Bash` kernel

<img src="img/R.png" alt="R" width="80"> Shift to the `popgen course` kernel
* You can run the code in each cell by clicking on the run cell sign in the toolbar, or simply by pressing <kbd>Shift</kbd>+<kbd>Enter</kbd>. When the code is done running, a small green check sign will appear on the left side.
* You need to run the cells in sequential order to execute the analysis. Please do not run a cell until the one above is done running, and do not skip any cells
* The code goes along with textual descriptions to help you understand what happens. Please try not to focus on understanding the code itself in too much detail - but rather try to focus on the explanations and on the output of the commands 
*   You can create new code cells by pressing `+` in the Menu bar above or by pressing <kbd>B</kbd> after you select a cell. 


## Learning outcomes

At the end of this tutorial you will be able to

- **handle and understand** the various steps needed to prepare data for alignment using standard tools like `samtools`
- **apply** genome alignment tools (`burrows-wheeler aligner BWA` and `Minimap2`) and **comment** their outcome
- **make sense** of the various data formats (`fasta`, `fastq`, `sam/bam`, `vcf`, ...) 
- **explore and analyze** data on a genome viewer

## Mapping reads against the reference

The first step when dealing with raw reads is mapping (aligning) them to a reference sequence. We will use two different aligners for mapping the reads against the reference genome; one is the classical Burrows-Wheeler aligner (BWA), and the other is the newer and faster Minimap2. Minimap2 is especially faster (up to 50X faster) and as accurate as BWA in aligning long reads, but works also well with short read data like ours.

Two input files are needed to do genome mapping:

- Fasta file containing your reference genome Which was downloaded from [hg19](https://hgdownload.soe.ucsc.edu/goldenPath/hg19/chromosomes/chr2.fa.gz). We need to copy the file from our `Data` folder, since it is a protected folder where we cannot write any file.
- The reads in fastq format, which can be found in `Data/fastq/`. Here we will not need to copy the files to another folder, because we will only read them.

We create a link to `Data`, so it is easier to access it. Then we make a folder called `Reference` and copy the file `chr2.fa` in it. Finally we create a folder where to save our results.

<img src="img/bash.png" alt="Bash" width="80"> Choose the `Bash` kernel

In [ ]:
mkdir -p ../../Reference
cp ../../Data/fasta/chr2.fa ../../Reference/
ln -s ../../Data
ln -s ../../Reference
mkdir -p Results

### Alignment with BWA

BWA allows for fast and accurate alignment of short reads to an indexed reference sequence. [Here](http://bio-bwa.sourceforge.net/bwa.shtml) is the manual. We will focus on a 10 MB region of chromosome 2 (from 135MB to 145MB), which contains the [lactase gene](https://www.genecards.org/cgi-bin/carddisp.pl?gene=LCT).

First we need to index the reference file for later use. This creates index files used to perform the alignment. To produce these files, run the following command. Here, `-a bwtsw` specifies the indexing algorithm; specifically `bwtsw` which is capable of handling large genomes such as the human genome (the standard option is `-a is`, which cannot handle beyond 2MB genomes). 

In [ ]:
bwa index -a bwtsw Reference/chr2.fa

The resulting `bwa` index files are in the same folder `Reference` as the reference genome file.

In [ ]:
ls Reference


We also need to generate a fasta file index, which contains the genomic coordinates of the reference file. This can be done using the `samtools` program `faidx`. Again, the resulting index files are in the `Reference` folder:

In [ ]:
samtools faidx Reference/chr2.fa

In [ ]:
ls Reference

Now you can map the reads to the reference. This will take a few minutes. Note how the alignment command `bwa mem` is piped with the symbol `|` at the end of the command. This allows the output of the alignment with `bwa mem` not to be written anywhere, but to be transferred (piped) directly into the following command `samtools sort`, which sorts the aligned genome by coordinates and creates a file in `bam` format. The sorting is necessary as the output of `bwa mem` is unsorted, and sorted files are usually the standard for genomic analysis. The piping speeds up the running time of the commands, avoiding to write intermediate files, and thus saving also disk space.

In [ ]:
bwa mem -t 16 -p Reference/chr2.fa './Data/fastq/S_Ju_hoan_North-3.region.fq' | samtools sort -O BAM -o './Results/S_Ju_hoan_North-3.bam'

Bam files follow this structure, where each element is tab-separated inside the output file:

![Alt text](https://us.v-cdn.net/5019796/uploads/editor/f4/uuzmf2cbau1y.png)

- Read name: ID for the given read.
- Flags: Combination of bitwise FLAGs that provide information on how the read is mapped  [Extra information](https://broadinstitute.github.io/picard/explain-flags.html).
- Position: Chromosome and position of the first base in the alignment. 
- MAPQ: Probability of wrong mapping of the read. It's in Phred scale, so higher numbers mean lower probabilities:
![Alt_text](https://genome.sph.umich.edu/w/images/math/e/9/d/e9dc88d1834c4579de12153a67ac3afa.png)
- CIGAR: summary of the alignment, including start position on the reference sequence, matches, mismatches, deletions and insertions. It may also include information on soft/hard clipping, i.e bases in the 3' and 5' ends of teh read that are not part of the alignment.
[Extra information](https://wiki.bits.vib.be/index.php/CIGAR)
- Mate information: chromosome and start position of teh read pair, and inferred insert size.
- Quality scores: base qualities of the read.
- Metadata: optional extra information. 

For more information, read [this](https://samtools.github.io/hts-specs/SAMv1.pdf)

Have a look at the bam file generated and compare it with the structure above. There is an error message, but do not mind it.

In [ ]:
samtools view ./Results/S_Ju_hoan_North-3.bam | head -n5

You can get useful statistics using `samtools flagstat`

In [ ]:
samtools flagstat ./Results/S_Ju_hoan_North-3.bam

### Alignment with Minimap2

[Minimap2](https://arxiv.org/abs/1708.01492) is a versatile sequence alignment program that aligns DNA or mRNA sequences against a large reference database. Typical use cases include: (1) mapping PacBio or Oxford Nanopore genomic reads to the human genome; (2) finding overlaps between long reads with error rate up to ~15%; (3) splice-aware alignment of PacBio Iso-Seq or Nanopore cDNA or Direct RNA reads against a reference genome; **(4) aligning Illumina single- or paired-end reads;** (5) assembly-to-assembly alignment; (6) full-genome alignment between two closely related species with divergence below ~15%. For >100bp Illumina short reads, minimap2 is three times as fast as BWA-MEM and Bowtie2, and as accurate on simulated data.

We apply `minimap2` to the same data used for `bwa`. Here, the option `-x sr` is used to select a preset for a specific type of data. In our case, `sr` denotes *Short single-end reads without splicing*. More information about other alignment scenarios can be found [in the manual](https://lh3.github.io/minimap2/minimap2.html).

In [ ]:
minimap2 -a -x sr Reference/chr2.fa Data/fastq/S_Ju_hoan_North-3.region.fq | \
                  samtools sort -O BAM -o './Results/S_Ju_hoan_North-3.minimap.bam'

Look at the output on the screen from `bwa mem` and `minimap2`. Both the real time (the actual time you wait for the command to run) and the CPU time (how much time in total multiple CPUs have been working while you waited for the real time) should be lower for `minimap2`, since this is in general faster than `bwa mem`.

## Explore the results with the IGV software

To visualize the alignment of the reads to the reference genome we have just produced, we will use the Integrative Genome Visualizer IGV. You can run it from your laptop after [downloading it](https://software.broadinstitute.org/software/igv/download), or running it through uCloud from the [Genomics Sandbox application](https://cloud.sdu.dk/app/jobs/create?app=genomics&version=2022.08.01) (only if you are an uCloud user). But first, we need to generate an index file for this software to work. Indexing a genome sorted BAM file allows one to quickly extract alignments overlapping particular genomic regions. Moreover, indexing is required by genome viewers such as IGV so that the viewers can quickly display alignments in each genomic region to which you navigate.

In [ ]:
samtools index ./Results/S_Ju_hoan_North-3.bam

IGV is an Integrative Genomics viewer and can be very useful to look at the results of Mapping and SNP calling. Three files are necessary to look at this dataset: the reference sequence and the
`.bam` and `.bai` files in the `Results` folder. Since we are using a human reference, the sequence is already available in the software and does not need a download. Download the `.bam` and `.bai` files on your laptop, and if you want to use uCloud, upload them into one of your personal folders.

### Reading in the data

To select the reference genome, go to `Genomes ---> Load from server... ---> Filter by human` and choose the `Human hg19` reference.

Now, you want to see the chromosomes and genes: download the mapping results by going to `File ---> Load from File... ----> S_Ju_hoan_North-3.bam`.

When you zoom in to the lactase region on chromosome 2 (chr2:136,545,410-136,594,750), you will see something like this: ![](img/IGV_example.png)

Try to understand what are the different attributes present in the viewer. If you zoom in very much you will find single nucleotide polymorphisms (SNPs), where the reference sequence does not have the same nucleotide as the data mapped to.

## Analyzing read coverage

One of the attributes one could learn from mapping reads back to the reference is the coverage of reads across the genome. In order to calculate the coverage depth you can use the command **samtools depth**.

In [ ]:
samtools depth Results/S_Ju_hoan_North-3.bam > Results/S_Ju_hoan_North-3.coverage

In [ ]:
less Results/S_Ju_hoan_North-3.coverage | head

<img src="img/R.png" alt="R" width="80"> We now switch to the `popgen course` kernel and use the `R` language to look at the depth file

In [ ]:
library(ggplot2)
library(dplyr)

In [ ]:
scaf <- read.table("./Results/S_Ju_hoan_North-3.coverage", header=FALSE, sep="\t", 
                   na.strings="NA", dec=".", strip.white=TRUE,
                   col.names = c("Scaffold", "locus", "depth"))
    
head(scaf)

We plot the average coverage in windows of 100 loci and save the plot in `Results/S_Ju_hoan_North-3.coverage.pdf`

In [ ]:
# Average in windows
scaf %>% 
mutate(rounded_position = round(locus, -2)) %>%
    group_by(rounded_position) %>% 
        summarize(mean_cov = mean(depth)) -> compressed

# Plotting the data
p <- ggplot(data =  compressed, aes(x=rounded_position, y=mean_cov)) + geom_area() + theme_classic() + ylim(0, 400)

# Saving your coverage plot
ggsave("Results/S_Ju_hoan_North-3.coverage.pdf",p)

Look at the pdf with the plot of the coverage. What are the conclusions you can extract from these analysis? Does the coverage match with what you observed with IGV? Does it match with what you would expect, i.e what you know from the data? 

## SNP calling

Even though just a tiny portion (around 2%) of our genomes are based of protein coding regions, this partition contains most of the disease causal variants (mutations), and that is why variant calling is so important from a medical point of view. From the population genetics side, it is also possible to use these variants to establish differences between individuals, populations and species. It can also be used to clarify the genetic basis of adaptation.

Once we have mapped our reads we can now start with variant detection. We will use the [software `bcftools`](https://samtools.github.io/bcftools/howtos/index.html), a program for variant calling and manipulating files in the Variant Call Format (VCF) and its binary counterpart BCF.

You will be using the VCF format further in the course, for now let's just count the number of heterozygous SNPs in each individual:

<img src="img/bash.png" alt="Bash" width="80"> Choose the `Bash` kernel

In [ ]:
bcftools mpileup --threads 16 -Ou --skip-indels -f Reference/chr2.fa Results/S_Ju_hoan_North-3.bam | \
         bcftools call -mv -Ov -o Results/S_Ju_hoan_North-3.vcf

The output is a single [VCF](http://samtools.github.io/hts-specs/VCFv4.2.pdf) file containing all the variants that `bcftools` identified.
Look at the output vcf file. What does the format look like? Does that match with what you observed in the IGV? Download the VCF file to the IGV browser to look atthe SNPs.

Here we can count how many line the file contains

In [ ]:
less ./Results/S_Ju_hoan_North-3.vcf | wc -l

Note that part of those lines (in this case 28) are just descriptions of the meaning of each column.

In [ ]:
less ./Results/S_Ju_hoan_North-3.vcf | head -n 30

The called genotype is written in the last column (together with other values) and can be

-   0/0 - the sample is homozygous to the reference (note that these sites usually won't be listed in single sample vcf files as they are not variants)
-   0/1 OR 0/1 - the sample is heterozygous, carrying 1 copy of each of the REF
    and ALT alleles
-   1/1 - the sample is homozygous for the alternate allele

We can count how many of those different called genotype there are by using `grep`. For the homozigous alternate allele,

In [ ]:
grep '1/1' ./Results/S_Ju_hoan_North-3.vcf | wc -l

For the homozigous genotypes

In [ ]:
grep '0/1\|1/0' ../../Results/S_Ju_hoan_North-3.vcf | wc -l

<img src="img/R.png" alt="R" width="80"> Choose the `popgen course` kernel

We can read in the vcf file in `R`

In [ ]:
t = read.table("../../Results/S_Ju_hoan_North-3.vcf",header=FALSE, sep="\t", na.strings="NA", dec=".", strip.white=TRUE)
    
head(t)

Choose column number 10

In [ ]:
geno_column = t[['V10']]

In [ ]:
geno_column[1:10]

For each element in the vector, choose only the part of text before `:`, so that the genotype is isolated from other values

In [ ]:
geno = sapply(geno_column, function(x) unlist(strsplit(x,':'))[1])

In [ ]:
geno[1:10]

Given this information you are now able to estimate the mean heterozygosity for your individual of the 10 MB region in chromosome 2.

In [ ]:
1 - (sum(geno=='0/1')/length(geno))^2 - (sum(geno=='1/1')/length(geno))^2